# LRFR Project - Google Colab Training Notebook

This notebook trains the Low-Resolution Facial Recognition (LRFR) models on Google Colab using an A100 GPU.

**Workflow:**
1.  **Code:** Cloned from GitHub (latest version).
2.  **Data:** Loaded from `dataset.zip` on Google Drive (fast I/O).

## Prerequisites
1.  **Zip Dataset:** Zip your `technical/dataset` folder into `dataset.zip` and upload it to your Google Drive (e.g., in a `LRFR_Project` folder).
2.  **Runtime:** Change runtime to **GPU (A100)**.

## 1. Setup Environment

In [ ]:
# Mount Google Drive (for dataset and saving results)
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
import shutil

# --- CONFIGURATION ---
GITHUB_REPO = 'https://github.com/dszurek/LRFR-Project.git'
PROJECT_ROOT = '/content/LRFR-Project'
# Path to dataset.zip on Drive. Adjust this if you put it in a subfolder.
DRIVE_ZIP_PATH = '/content/drive/MyDrive/LRFR_Project/dataset.zip'
# ---------------------

# Clone Repository
if os.path.exists(PROJECT_ROOT):
    print("Repository already exists. Pulling latest changes...")
    os.chdir(PROJECT_ROOT)
    !git pull
else:
    print("Cloning repository...")
    os.chdir('/content')
    !git clone {GITHUB_REPO}

os.chdir(PROJECT_ROOT)
sys.path.append(PROJECT_ROOT)
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Install dependencies from requirements.txt
%pip install -r requirements.txt

## 2. Data Preparation

We unzip `dataset.zip` from Drive to the local VM for high-speed training.

In [ ]:
# Check for dataset zip on Drive
if not os.path.exists(DRIVE_ZIP_PATH):
    # Try looking in root if not found in folder
    DRIVE_ZIP_PATH_ALT = '/content/drive/MyDrive/dataset.zip'
    if os.path.exists(DRIVE_ZIP_PATH_ALT):
        DRIVE_ZIP_PATH = DRIVE_ZIP_PATH_ALT
        print(f"Found dataset.zip at root: {DRIVE_ZIP_PATH}")
    else:
        print(f"❌ ERROR: dataset.zip not found at {DRIVE_ZIP_PATH} or {DRIVE_ZIP_PATH_ALT}")
        print("Please upload dataset.zip to your Google Drive.")

if os.path.exists(DRIVE_ZIP_PATH):
    print(f"Unzipping {DRIVE_ZIP_PATH} to local VM...")
    
    # Unzip to /content/temp_dataset first to inspect structure
    temp_extract = '/content/temp_dataset'
    if os.path.exists(temp_extract):
        shutil.rmtree(temp_extract)
    shutil.unpack_archive(DRIVE_ZIP_PATH, temp_extract)
    
    # Move to expected location: PROJECT_ROOT/technical/dataset
    target_dir = os.path.join(PROJECT_ROOT, 'technical', 'dataset')
    os.makedirs(target_dir, exist_ok=True)
    
    # Check if it unzipped as 'dataset' folder or contents directly
    if os.path.exists(os.path.join(temp_extract, 'dataset')):
        source = os.path.join(temp_extract, 'dataset')
    elif os.path.exists(os.path.join(temp_extract, 'train_processed')):
        source = temp_extract
    elif os.path.exists(os.path.join(temp_extract, 'technical', 'dataset')):
        source = os.path.join(temp_extract, 'technical', 'dataset')
    else:
        source = temp_extract # Hope for the best or it's a flat structure
        
    # Move contents
    for item in os.listdir(source):
        s = os.path.join(source, item)
        d = os.path.join(target_dir, item)
        if os.path.exists(d):
            if os.path.isdir(d):
                shutil.rmtree(d)
            else:
                os.remove(d)
        shutil.move(s, d)
        
    print(f"✅ Dataset ready at {target_dir}")
    shutil.rmtree(temp_extract)

## 3. Training

### 3.1 Train Hybrid DSR Models
Train the Hybrid Transformer-CNN Super-Resolution models.

In [ ]:
# Train Hybrid DSR (16, 24, 32)
!python -m technical.dsr.train_hybrid_dsr --vlr-size 16
!python -m technical.dsr.train_hybrid_dsr --vlr-size 24
!python -m technical.dsr.train_hybrid_dsr --vlr-size 32

### 3.2 Fine-tune EdgeFace
Fine-tune the face recognition model for each resolution.

In [ ]:
# Fine-tune EdgeFace
!python -m technical.facial_rec.finetune_edgeface --vlr-size 16 --device cuda
!python -m technical.facial_rec.finetune_edgeface --vlr-size 24 --device cuda
!python -m technical.facial_rec.finetune_edgeface --vlr-size 32 --device cuda

## 4. Evaluation & Saving Results

In [ ]:
!python -m technical.pipeline.evaluate_cli --resolutions 16 24 32 --output-dir evaluation_results

In [ ]:
# Save results back to Drive
import glob
SAVE_DIR = '/content/drive/MyDrive/LRFR_Project/results'
os.makedirs(SAVE_DIR, exist_ok=True)

print(f"Saving models and results to {SAVE_DIR}...")

# Copy DSR models (both standard and hybrid naming patterns just in case)
for f in glob.glob('technical/dsr/*.pth'):
    shutil.copy(f, SAVE_DIR)

# Copy EdgeFace weights
for f in glob.glob('technical/facial_rec/edgeface_weights/*.pth'):
    shutil.copy(f, SAVE_DIR)

# Copy Evaluation Results
if os.path.exists('evaluation_results'):
    eval_dest = os.path.join(SAVE_DIR, 'evaluation_results')
    if os.path.exists(eval_dest):
        shutil.rmtree(eval_dest)
    shutil.copytree('evaluation_results', eval_dest)

print("✅ All results saved to Drive!")